In [1]:
import cv2  # openvv 读取的格式是BGR
import matplotlib.pyplot as plt
import numpy as np
# 当调用 matplotlib.pyplot 的绘图函数 plot() 进行绘图的时候，或者生成一个 figure 画布的时候，可以直接在你的 python console 里面生成图像
%matplotlib inline  

In [2]:
# 可以定义一个方法专门用于显示图像
def cv_show(name, img):
    '''
    :param name: 窗口名字
    :param img: 显示的图像
    '''
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()